In [1]:
from sklearn.metrics import confusion_matrix
import datetime as dt
import numpy as np
import pandas as pd

path = ""

In [2]:
#load the labels
test_label = [i.strip() for i in open(path+"test-label.dat").readlines()]
test_label = np.array(test_label)
train_label = [i.strip() for i in open(path+"train-label.dat").readlines()]
train_label = np.array(train_label)

#load the data 3 times (for each classifier,)
dfKNN = pd.read_csv("train_kmedoids.csv") 
df_testKNN = pd.read_csv("test_kmedoids.csv")
dfSVM = pd.read_csv("train_kmedoids.csv") 
df_testSVM = pd.read_csv("test_kmedoids.csv")
dfRF= pd.read_csv("train_kmedoids.csv") 
df_testRF = pd.read_csv("test_kmedoids.csv")

#shape the labels by label (each row represents one label) 
labels_models=[]
for i in range (0,20 ):
    tr=[]
    for j in range (0,len(train_label)):
        tr.append(int(train_label[j][2*i]))
    labels_models.append(tr)
    
train_labels = pd.DataFrame(labels_models)
train_labels = train_labels.transpose()

# test labels
labels__models=[]
for i in range (0,20 ):
    ts=[]
    for j in range (0,len(test_label)):
        ts.append(int(test_label[j][2*i]))
    labels__models.append(ts)
    
test_labels = pd.DataFrame(labels__models)
test_labels = test_labels.transpose()


#normalize the data
from sklearn import preprocessing
x1 = dfKNN.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled1 = min_max_scaler.fit_transform(x1)
dfKNN = pd.DataFrame(x_scaled1)
x1 = df_testKNN.values #returns a numpy array
x_scaled1 = min_max_scaler.transform(x1)
df_testKNN = pd.DataFrame(x_scaled1)
dfinitKNN = dfKNN.copy()



x2 = dfSVM.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled2 = min_max_scaler.fit_transform(x2)
dfSVM = pd.DataFrame(x_scaled2)
x2 = df_testSVM.values #returns a numpy array
x_scaled2 = min_max_scaler.transform(x2)
df_testSVM = pd.DataFrame(x_scaled2)
dfinitSVM = dfSVM.copy()


x3 = dfRF.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled3 = min_max_scaler.fit_transform(x3)
dfRF = pd.DataFrame(x_scaled3)
x3 = df_testRF.values #returns a numpy array
x_scaled3 = min_max_scaler.transform(x3)
df_testRF = pd.DataFrame(x_scaled3)
dfinitRF = dfRF.copy()



In [3]:
def multilabel_confusion_matrix(predict,path,clustnum=-3):
    test_label = [i.strip().split() for i in open(path + "test-label.dat").readlines()]

    for i in range(len(test_label)):
        item = test_label[i]
        for j in range(len(item)):
            item[j] = int(item[j])

        test_label[i] = item

    labels = []

    for item in open(path + "labels.txt").readlines():
        item = item.replace('\n', '')
        label_name, _ = item.split(', ')
        labels.append(label_name)

    y_true = np.array(test_label)
    y_pred = np.array(predict)

    conf_mat_dict = {}

    for label_col in range(len(labels)):
        y_true_label = y_true[:, label_col]
        y_pred_label = y_pred[:, label_col]
        conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)
    f = open("confmatrix"+str(clustnum)+".txt", "w")
    f.close()
    for label, matrix in conf_mat_dict.items():
        print("Confusion matrix for label {}:".format(label))
        f = open("confmatrix"+str(clustnum)+".txt", "a")
        print(matrix)
        f.write("Confusion matrix for label {}:".format(label))
        np.savetxt(f,matrix)
        f.close()
    return

In [4]:
# apply a combination of oversampling and undersampling technique
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
sm = SMOTE(sampling_strategy=0.5, random_state=2)
smsmall = SMOTE(sampling_strategy=0.25, random_state=2)
rm = RandomUnderSampler(random_state=42)


# Fit an ensemble of 10 classifier chains and take the average prediction of all the chains.

orderlist1 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
#orderlist3 = [1,3,15,5,7,9,11,2,4,13,6,18,8,17,10,14,12,19,16,0]
#orderlist4 = [2,4,17,6,8,3,13,19,9,10,14,1,12,0,16,5,7,11,15,18]
#orderlist5 = [19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]
#orderlist6 = [4,6,3,15,11,7,10,1,13,5,0,12,16,2,18,9,17,8,14,19]
#orderlist7 = [15,6,8,0,3,13,18,10,12,1,4,2,17,14,16,11,9,7,19,5]
#orderlist8 = [12,7,4,14,16,6,0,19,10,8,3,5,1,9,2,15,18,11,13,17]
#orderlist9 = [5,8,19,10,6,12,16,0,3,11,14,4,7,9,17,15,18,13,2,1]
#orderlist10 =[19,18,1,16,13,3,14,15,11,17,9,10,12,5,6,8,0,4,7,2]
OrderList = [orderlist1]#, orderlist2, orderlist3, orderlist4, orderlist5, orderlist6, orderlist7, orderlist8, orderlist9, orderlist10]


# this function ordering the predictions of every chain in the same way
# to fit the confusion matrix correctly
def ordering_labels(order,predictions):
    new_predictions = []
    for pred in predictions:
        temp_predict = [0] * len(pred) 
        for i in range(0,len(pred)):
            temp_predict[ order[i] ] = pred[i]
        new_predictions.append(temp_predict)
    return new_predictions   

In [5]:
###############################################################################
#.................................KNN..........................................
###############################################################################
from sklearn import neighbors
predKNN = pd.DataFrame()
KNNpredictions= []
start_time = dt.datetime.now()
for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfKNN, train_labels_new = smsmall.fit_resample(dfinitKNN,train_labels[:][l])
            dfKNN, train_labels_new = rm.fit_resample(dfKNN,train_labels_new)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfKNN, train_labels_new = sm.fit_resample(dfinitKNN,train_labels[:][l])
            dfKNN, train_labels_new = rm.fit_resample(dfKNN,train_labels_new)
        else:
            dfKNN, train_labels_new = rm.fit_resample(dfinitKNN, train_labels[:][l])
        #print(dfKNN.shape)
        KNN = neighbors.KNeighborsClassifier(n_neighbors=3 ,weights='distance',metric='minkowski',p=1).fit(dfKNN, train_labels_new)
        dfinitKNN['Label'+str(l)] = train_labels[:][l]
        df_testKNN['Label'+str(l)] = KNN.predict(df_testKNN) 
        predKNN['Label'+str(l)] = df_testKNN['Label'+str(l)]


    #predKNN = KNN.predict(df_test)
    #print(predKNN)
    predictKNN=[]
    for p in range(3983):
        #print(p)
        resultKNN = []
        for r in range(20):
            x = orderlist.index(r)
            resultKNN.append(predKNN.iloc[p,x])
        predictKNN.append(resultKNN)
    KNNpredictions.append(predictKNN)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)
#multilabel_confusion_matrix(predict,path)

the training time is: 0:00:10.171410


In [6]:
final_predictionsKNN = []
for i in range(0,len(KNNpredictions)):
    final_predictionsKNN.append(ordering_labels( OrderList[i], KNNpredictions[i]))


In [7]:
FinalKNN = []
for i in range(0,len(final_predictionsKNN[0])):
    tempKNN = [0] * len(final_predictionsKNN[0][0])
    for j in range(0,len(final_predictionsKNN)):
        tempKNN = [tempKNN[k] + final_predictionsKNN[j][i][k] for k in range(len(tempKNN))]
    tempKNN = [int(x / len(final_predictionsKNN) + 0.5) for x in tempKNN]
    FinalKNN.append(tempKNN)
    

In [8]:
#...........KNN..................
#confusion matrix for every label
print('for KNN classifier:')
multilabel_confusion_matrix(FinalKNN,path)


for KNN classifier:
Confusion matrix for label programming:
[[1744 1262]
 [ 423  554]]
Confusion matrix for label style:
[[2521 1234]
 [ 143   85]]
Confusion matrix for label reference:
[[1206 1219]
 [ 726  832]]
Confusion matrix for label java:
[[2296 1315]
 [ 193  179]]
Confusion matrix for label web:
[[1552 1381]
 [ 464  586]]
Confusion matrix for label internet:
[[2451  995]
 [ 362  175]]
Confusion matrix for label culture:
[[2458  823]
 [ 479  223]]
Confusion matrix for label design:
[[1239 1665]
 [ 439  640]]
Confusion matrix for label education:
[[2623  557]
 [ 634  169]]
Confusion matrix for label language:
[[2734  766]
 [ 386   97]]
Confusion matrix for label books:
[[2610  866]
 [ 383  124]]
Confusion matrix for label writing:
[[2592  913]
 [ 347  131]]
Confusion matrix for label computer:
[[2583  891]
 [ 348  161]]
Confusion matrix for label english:
[[3070  558]
 [ 306   49]]
Confusion matrix for label politics:
[[3064  527]
 [ 301   91]]
Confusion matrix for label history:

In [9]:
#.......KNN.......
#Balanced Accuracy
FinalKNN=pd.DataFrame(FinalKNN)
from sklearn.metrics import balanced_accuracy_score
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print('for kNN classifier')
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalKNN[i]))
    
   


for kNN classifier
the Balanced Accuracy for the labels 
programming : 0.5736074762791034
style : 0.5220892611021561
reference : 0.5156687796937655
java : 0.5585088722332896
web : 0.5436231389930675
internet : 0.5185719874931235
culture : 0.5334128292829908
design : 0.5098973452615292
education : 0.5176517697001026
language : 0.4909855072463768
books : 0.49771949893663625
writing : 0.5067867780039275
computer : 0.5299148996813827
english : 0.49211221019612716
politics : 0.5426935394040657
history : 0.5084541062801933
philosophy : 0.5513509617983197
science : 0.5144899746744306
religion : 0.5165250399164825
grammar : 0.4840874914559125


In [10]:
###############################################################################
#.................................SVM..........................................
###############################################################################

from sklearn import svm
predSVM = pd.DataFrame()

SVMpredictions= []
start_time = dt.datetime.now()

for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfSVM, train_labels_new2 = smsmall.fit_resample(dfinitSVM,train_labels[:][l])
            dfSVM, train_labels_new2 = rm.fit_resample(dfSVM,train_labels_new2)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfSVM, train_labels_new2 = sm.fit_resample(dfinitSVM,train_labels[:][l])
            dfSVM, train_labels_new2 = rm.fit_resample(dfSVM,train_labels_new2)
        else:
            dfSVM, train_labels_new2 = rm.fit_resample(dfinitSVM, train_labels[:][l])
        #print(df.shape)
        SVM= svm.SVC(C=10 ,gamma=1.5, kernel='rbf' , random_state=41).fit(dfSVM, train_labels_new2)
        dfinitSVM['Label'+str(l)] = train_labels[:][l]
        df_testSVM['Label'+str(l)] = SVM.predict(df_testSVM) 
        predSVM['Label'+str(l)] = df_testSVM['Label'+str(l)]


    
    predictSVM=[]
    for p in range(3983):
        #print(p)
        resultSVM = []
        for r in range(20):
            x = orderlist.index(r)
            resultSVM.append(predSVM.iloc[p,x])
        predictSVM.append(resultSVM)
    SVMpredictions.append(predictSVM)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)

the training time is: 0:00:36.589899


In [11]:
final_predictionsSVM = []
for i in range(0,len(SVMpredictions)):
    final_predictionsSVM.append(ordering_labels( OrderList[i], SVMpredictions[i]))
    
FinalSVM = []
for i in range(0,len(final_predictionsSVM[0])):
    tempSVM = [0] * len(final_predictionsSVM[0][0])
    for j in range(0,len(final_predictionsSVM)):
        tempSVM = [tempSVM[k] + final_predictionsSVM[j][i][k] for k in range(len(tempSVM))]
    tempSVM = [int(x / len(final_predictionsSVM) + 0.5) for x in tempSVM]
    FinalSVM.append(tempSVM)
    

In [12]:
#...........SVM..................
#confusion matrix for every label
print("for SVM classifier:")
multilabel_confusion_matrix(FinalSVM,path)


for SVM classifier:
Confusion matrix for label programming:
[[1772 1234]
 [ 347  630]]
Confusion matrix for label style:
[[2415 1340]
 [ 145   83]]
Confusion matrix for label reference:
[[1245 1180]
 [ 671  887]]
Confusion matrix for label java:
[[2285 1326]
 [ 181  191]]
Confusion matrix for label web:
[[1919 1014]
 [ 552  498]]
Confusion matrix for label internet:
[[3011  435]
 [ 470   67]]
Confusion matrix for label culture:
[[2351  930]
 [ 444  258]]
Confusion matrix for label design:
[[1052 1852]
 [ 396  683]]
Confusion matrix for label education:
[[2750  430]
 [ 666  137]]
Confusion matrix for label language:
[[2894  606]
 [ 395   88]]
Confusion matrix for label books:
[[2966  510]
 [ 430   77]]
Confusion matrix for label writing:
[[2945  560]
 [ 415   63]]
Confusion matrix for label computer:
[[2947  527]
 [ 398  111]]
Confusion matrix for label english:
[[3135  493]
 [ 298   57]]
Confusion matrix for label politics:
[[3120  471]
 [ 318   74]]
Confusion matrix for label history:

In [13]:
#.......SVM.......
#Balanced Accuracy
FinalSVM=pd.DataFrame(FinalSVM)
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalSVM[i]))
    

the Balanced Accuracy for the labels 
programming : 0.6171594034721414
style : 0.5035887822085173
reference : 0.5413608512102484
java : 0.5731147807029298
web : 0.5642823048073644
internet : 0.4992669556693265
culture : 0.542035599944774
design : 0.49762623283981444
education : 0.5176950429599693
language : 0.5045258799171842
books : 0.5025766995095136
writing : 0.4860137042718412
computer : 0.533188162866899
english : 0.5123379194682982
politics : 0.5288071368898436
history : 0.5201056707267888
philosophy : 0.5246635357423833
science : 0.521554822577705
religion : 0.5078355287398674
grammar : 0.5026999316473001


In [14]:
###############################################################################
#..........................Random Forest.....................................
###############################################################################
from sklearn.ensemble import RandomForestClassifier
predRF = pd.DataFrame()
RFpredictions= []
start_time = dt.datetime.now()

      
      
for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfRF, train_labels_new3 = smsmall.fit_resample(dfinitRF,train_labels[:][l])
            dfRF, train_labels_new3 = rm.fit_resample(dfRF,train_labels_new3)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfRF, train_labels_new3 = sm.fit_resample(dfinitRF,train_labels[:][l])
            dfRF, train_labels_new3 = rm.fit_resample(dfRF,train_labels_new3)
        else:
            dfRF, train_labels_new3 = rm.fit_resample(dfinitRF, train_labels[:][l])
        #print(df.shape)
        Random_Forest = RandomForestClassifier(n_estimators=20, criterion='gini', random_state=41).fit(dfRF, train_labels_new3)
        dfinitRF['Label'+str(l)] = train_labels[:][l]
        df_testRF['Label'+str(l)] = Random_Forest.predict(df_testRF) 
        predRF['Label'+str(l)] = df_testRF['Label'+str(l)]


    
    predictRF=[]
    for p in range(3983):
        #print(p)
        resultRF = []
        for r in range(20):
            x = orderlist.index(r)
            resultRF.append(predRF.iloc[p,x])
        predictRF.append(resultRF)
    RFpredictions.append(predictRF)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)  

the training time is: 0:00:07.401730


In [15]:
final_predictionsRF = []
for i in range(0,len(RFpredictions)):
    final_predictionsRF.append(ordering_labels( OrderList[i], RFpredictions[i]))
    
FinalRF = []
for i in range(0,len(final_predictionsRF[0])):
    tempRF = [0] * len(final_predictionsRF[0][0])
    for j in range(0,len(final_predictionsRF)):
        tempRF = [tempRF[k] + final_predictionsRF[j][i][k] for k in range(len(tempRF))]
    tempRF = [int(x / len(final_predictionsRF) + 0.5) for x in tempRF]
    FinalRF.append(tempRF)
    

In [16]:
#........Random Forest...........
#confusion matrix for every label
print("for Random Forest classifier:")
multilabel_confusion_matrix(FinalRF,path)

for Random Forest classifier:
Confusion matrix for label programming:
[[1961 1045]
 [ 413  564]]
Confusion matrix for label style:
[[3173  582]
 [ 179   49]]
Confusion matrix for label reference:
[[1444  981]
 [ 829  729]]
Confusion matrix for label java:
[[2526 1085]
 [ 182  190]]
Confusion matrix for label web:
[[1801 1132]
 [ 471  579]]
Confusion matrix for label internet:
[[2874  572]
 [ 436  101]]
Confusion matrix for label culture:
[[2486  795]
 [ 428  274]]
Confusion matrix for label design:
[[1398 1506]
 [ 402  677]]
Confusion matrix for label education:
[[2688  492]
 [ 621  182]]
Confusion matrix for label language:
[[3087  413]
 [ 382  101]]
Confusion matrix for label books:
[[3069  407]
 [ 440   67]]
Confusion matrix for label writing:
[[2889  616]
 [ 360  118]]
Confusion matrix for label computer:
[[2969  505]
 [ 410   99]]
Confusion matrix for label english:
[[3156  472]
 [ 279   76]]
Confusion matrix for label politics:
[[2979  612]
 [ 272  120]]
Confusion matrix for labe

In [17]:
#..Random Forest..
#Balanced Accuracy
FinalRF=pd.DataFrame(FinalRF)
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalRF[i]))
    

the Balanced Accuracy for the labels 
programming : 0.6148196612574919
style : 0.5299594692456842
reference : 0.5316857456691767
java : 0.6051409522278105
web : 0.5827378111148994
internet : 0.5110461917901197
culture : 0.5740046073785787
design : 0.5544188834166928
education : 0.5359665405672126
language : 0.5455548654244307
books : 0.507530646892867
writing : 0.5355564972931676
computer : 0.5245667224275081
english : 0.5419926394086682
politics : 0.5678481919083422
history : 0.5435342780063277
philosophy : 0.5538217695327435
science : 0.5254462356160741
religion : 0.5421681814460002
grammar : 0.5405263157894737
